In [1]:
filtered_tokens = [(1, 'deep', 'JJ'), (2, 'learning', 'NN'), (3, 'methods', 'NNS'), (5, 'many', 'JJ'), (7, 'layers', 'NNS'), (11, 'stratified', 'JJ'), (12, 'representation', 'NN'), (14, 'data', 'NNS'), (18, 'state-of-art', 'JJ'), (19, 'results', 'NNS'), (21, 'several', 'JJ'), (22, 'domains', 'NNS'), (26, 'deep', 'JJ'), (27, 'learning', 'NN'), (28, 'model', 'NN'), (29, 'designs', 'NNS'), (31, 'architectures', 'NNS'), (36, 'context', 'NN'), (38, 'natural', 'JJ'), (39, 'language', 'NN'), (40, 'processing', 'NN'), (42, 'nlp', 'JJ'), (46, 'survey', 'NN'), (49, 'brief', 'JJ'), (50, 'description', 'NN'), (53, 'advances', 'NNS'), (59, 'area', 'NN'), (61, 'deep', 'JJ'), (62, 'generative', 'JJ'), (63, 'modeling', 'NN'), (66, 'work', 'NN'), (68, 'most', 'JJS'), (71, 'papers', 'NNS'), (74, 'onwards', 'NNS'), (78, 'paper', 'NN'), (82, 'many', 'JJ'), (83, 'deep', 'JJ'), (84, 'learning', 'NN'), (85, 'models', 'NNS'), (92, 'generation', 'NN'), (94, 'text', 'NN'), (100, 'various', 'JJ'), (101, 'models', 'NNS'), (107, 'detailed', 'JJ'), (108, 'understanding', 'NN'), (110, 'past', 'JJ'), (112, 'present', 'JJ'), (115, 'future', 'NN'), (117, 'text', 'JJ'), 
(118, 'generation', 'NN'), (119, 'models', 'NNS'), (121, 'deep', 'JJ'), (122, 'learning', 'NN'), (126, 'dl', 'JJ'), (127, 'approaches', 'NNS'), (135, 'different', 'JJ'), (136, 'application', 'NN'), (137, 'domains', 'NNS'), (139, 'nlp', 'NN'), (144, 'survey', 'NN')]

In [2]:
from collections import Counter

In [3]:
class PRGraph(object):
    def __init__(self, filtered_tokens, w, k) -> None:
        self.total_cnt = len(filtered_tokens)
        words = [t[1] for t in filtered_tokens]
        unique_words = list(Counter(words))
        self.unique_cnt = len(unique_words)
        self.conversion = {unique_words[i] : i for i in range(len(unique_words))}
        # self.V = {self.conversion[v] : 1 / self.unique_cnt for v in unique_words}
        self.S = [1 / self.unique_cnt for i in range(self.unique_cnt)]
        self.E = {self.conversion[v] : [] for v in unique_words}
        self.M = [[0 for _ in range(self.unique_cnt)] for _ in range(self.unique_cnt)]
        self.alpha = 0.85
        self.threshold = 0.001

        for i in range(self.total_cnt):
            token = filtered_tokens[i]
            for j in range(w):
                if i + j + 1 >= self.total_cnt:
                    break
                else:
                    next_token = filtered_tokens[i + j + 1]
                    if token[0] + w <= next_token[0]:
                        break
                    else:
                        idx = self.conversion[token[1]]
                        next_idx = self.conversion[next_token[1]]
                        if next_idx not in self.E[idx]:
                            self.E[idx].append(next_idx)
                            self.E[next_idx].append(idx)
                        self.M[idx][next_idx] += 1
                        self.M[next_idx][idx] += 1
        
        self.M_tilde = [[0 for j in range(self.unique_cnt)] for i in range(self.unique_cnt)]
        for i in range(self.unique_cnt):
            row_sum = sum(self.M[i])
            if row_sum != 0:
                for j in range(self.unique_cnt):
                    self.M_tilde[i][j] = self.M[i][j] / row_sum
        
        self.p_tilde = [0 for i in range(self.unique_cnt)]
        for i in range(self.total_cnt):
            token = filtered_tokens[i]
            idx = self.conversion[token[1]]
            self.p_tilde[idx] += 1 / token[0]
        row_sum = sum(self.p_tilde)
        for i in range(self.unique_cnt):
            self.p_tilde[i] /= row_sum

        self.conversion = {val : key for key, val in self.conversion.items()}
    
    def score_of(self, i):
        temp = 0
        for j in self.E[i]:
            # temp += self.M_tilde[i][j] / sum(self.M[j])
            temp += self.M[i][j] / sum(self.M[j])
        return (1 - self.alpha) * self.p_tilde[i] + self.alpha * temp * self.S[i]

    def calculate_positionrank(self):
        flags = [False for i in range(self.unique_cnt)]
        i = 0
        iter_cnt = 0
        while not all(flags) and iter_cnt < 100:
            prev_score = self.S[i]
            curr_score = self.score_of(i)
            self.S[i] = curr_score
            if abs(prev_score - curr_score) < self.threshold:
                flags[i] = True
            i = (i + 1) % self.unique_cnt
            if i == 0:
                iter_cnt += 1
        return iter_cnt

In [4]:
graph = PRGraph(filtered_tokens, 2, 5)

In [5]:
print(graph.total_cnt, graph.unique_cnt)

60 45


In [6]:
# graph.conversion

In [7]:
# graph.V

In [8]:
# graph.E

In [9]:
# graph.M_tilde

In [10]:
# graph.p_tilde

In [11]:
iter_cnt = graph.calculate_positionrank()
iter_cnt

100

In [12]:
# graph.S

In [13]:
kp_score = {}
for i in range(graph.unique_cnt):
    kp_score[graph.conversion[i]] = graph.S[i]
kp_score

{'deep': 3.1515487090927914e+23,
 'learning': 2.407961350946923e+31,
 'methods': 0.016863427251819472,
 'many': 0.01098812499709402,
 'layers': 0.006349596587674883,
 'stratified': 0.026937682080673098,
 'representation': 0.024692875402610955,
 'data': 0.0031747982938374413,
 'state-of-art': 0.016461917583049795,
 'results': 0.015595500970464412,
 'several': 0.0036809255580723954,
 'domains': 1093563589.1132123,
 'model': 0.053722387394820666,
 'designs': 0.0026654978179144933,
 'architectures': 0.0014337798746362638,
 'context': 0.0012346437809367828,
 'natural': 0.002034195703145271,
 'language': 2.644760922593623e+21,
 'processing': 0.001932485917988008,
 'nlp': 0.0013780299548790812,
 'survey': 0.0012749039042281997,
 'brief': 0.006047236260339756,
 'description': 0.0059262915740115095,
 'advances': 0.0008386259644098902,
 'area': 0.0007533419680292233,
 'generative': 0.05839409245583255,
 'modeling': 0.0012269751860241318,
 'work': 0.0006734420623291543,
 'most': 0.000653634942848

In [14]:
def get_top_n_grams(filtered_tokens, kp_score):
    kw = {}
    i = 0
    while i < len(filtered_tokens):
        token = filtered_tokens[i]
        curr_idx = token[0]
        curr_word = token[1]
        curr_score = kp_score[curr_word]
        # flag = True
        j = i + 1
        # while flag and j < len(filtered_tokens) and j < i + 3:
        while j < len(filtered_tokens) and j < i + 3:
            next_token = filtered_tokens[j]
            next_idx = next_token[0]
            if curr_idx + 1 != next_idx:
                # flag = False
                break
            else:
                curr_idx = next_idx
                curr_word += ' ' + next_token[1]
                curr_score += kp_score[next_token[1]]
                j += 1
        i = j
        # i += 1
        if curr_word not in kw:
            kw[curr_word] = curr_score
    kw = sorted(kw.items(), key=lambda x : x[1], reverse=True)
    return kw

In [15]:
kw = get_n_grams(filtered_tokens, kp_score)
kw = sorted(kw.items(), key=lambda x : x[1], reverse=True)
kw

[('deep learning methods', 2.4079613824624103e+31),
 ('deep learning model', 2.4079613824624103e+31),
 ('many deep learning', 2.4079613824624103e+31),
 ('deep learning', 2.4079613824624103e+31),
 ('deep generative modeling', 3.1515487090927914e+23),
 ('natural language processing', 2.644760922593623e+21),
 ('text generation models', 8128359064024.222),
 ('various models', 8128359056204.277),
 ('models', 8128359056204.276),
 ('different application domains', 1926155021.9931242),
 ('several domains', 1093563589.1168933),
 ('generation', 7819.944143761324),
 ('stratified representation', 0.05163055748328405),
 ('state-of-art results', 0.03205741855351421),
 ('brief description', 0.011973527834351266),
 ('many', 0.01098812499709402),
 ('layers', 0.006349596587674883),
 ('detailed understanding', 0.005512950714040772),
 ('dl approaches', 0.004684891015749152),
 ('data', 0.0031747982938374413),
 ('designs', 0.0026654978179144933),
 ('text', 0.0014830133795070725),
 ('architectures', 0.001433